# Langchain + ChromaDB -Q&A Multiple files

1. Multiple files
2. ChromaDB
3. gpt


In [ ]:
from sympy.physics.units import temperature
from tenacity import wait_chain
!pip -q install chromadb langchain langchain-community openai tiktoken

In [ ]:
!pip show langchain

In [ ]:
!pip install langchain-openai

In [ ]:
import os

In [ ]:
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [ ]:
!unzip -q techcrunch_articles.zip -d articles

In [ ]:
# load multiple documents and process documents

loader = DirectoryLoader("./articles", glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [ ]:
documents

In [ ]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
texts

In [ ]:
len(texts)

In [ ]:
# Create a ChromaDB
persist_directory = "db"
embedding = OpenAIEmbeddings()

In [ ]:
vectordb = Chroma.from_documents(
    documents = texts,
    embedding = embedding,
    persist_directory = persist_directory,
)

In [ ]:
# persist the db to the disk
vectordb.persist()
vectordb = None

In [ ]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
retriever.search_type

In [ ]:
retriever.search_kwargs

In [ ]:
#turbo_llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo")

# 4. Create LLM with new ChatOpenAI (instead of old OpenAI())
llm = ChatOpenAI( model="gpt-4.1-mini",temperature=0)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

In [ ]:
query = "What is the news about Pando?"
llm_response = qa_chain(query)

In [ ]:
def process_llm_response(llm_response):
    print(llm_response["result"])
    print("\n\nSources:")
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
process_llm_response(llm_response=llm_response)

In [ ]:
query = "what is the news about databrick?"
llm_response = qa_chain(query)
process_llm_response(llm_response=llm_response)